##### Code review.

##### Привет.
###### 1. Хорошие комментарии. 
###### 2. Структура кода хорошая step-by-step.
###### 3. Я обычно все библиотеки, которые точно применила, в header собираю по окончании. Так сразу видно, что используется (imho).

##### Хорошего дня (:

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "konstantin.ivanchenko Spark RDD app") 

sc = SparkContext(conf=conf)

In [129]:
! hdfs dfs -cat /labs/lab06data/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [10]:
# загружаем набор данных из файла
rdd = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [67]:
# проверяем структуру данных
rdd.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

### Формирование списка оценок для item_id = 195

In [88]:
# разделяем строки по табуляции и фильтруем по id фильма.
rdd_sep = rdd.map(lambda x: x.split("\t")).filter(lambda x: x[1] == '195')

In [116]:
# строим коллекцию пар (оценка, 1). Группируем по ключу (оценка). 
# Подсчитываем длину массива единиц, это и будет количество оценок. Упорядочиваем по ключу.
rdd_myfilm = rdd_sep.map(lambda x: (x[2], 1)).groupByKey().map(lambda x: (x[0], len(x[1]))).takeOrdered(5)

In [125]:
# Проверяем полученный агрегированный набор (оценка, количество оценок)
rdd_myfilm

[('1', 8), ('2', 14), ('3', 62), ('4', 123), ('5', 94)]

In [117]:
# формируем список количества оценок
hist_film = [v for k, v in rdd_myfilm]

In [118]:
# Проверка списка
hist_film

[8, 14, 62, 123, 94]

### Формирование общего списка оценок

In [120]:
# разделяем строки по табуляции.
rdd_sep_all = rdd.map(lambda x: x.split("\t")).cache()

In [121]:
# строим коллекцию пар (оценка, 1). Группируем по ключу (оценка). 
# Подсчитываем длину массива единиц, это и будет количество оценок. Упорядочиваем по ключу.
rdd_all = rdd_sep_all.map(lambda x: (x[2], 1)).groupByKey().map(lambda x: (x[0], len(x[1]))).takeOrdered(5)

In [124]:
# Проверяем полученный агрегированный набор (оценка, количество оценок)
rdd_all

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [122]:
# формируем список количества оценок
hist_all = [v for k, v in rdd_all]

In [123]:
# Проверка списка
hist_all

[6110, 11370, 27145, 34174, 21201]

In [126]:
import json

In [127]:
# Вывод данных в файл
response = {'hist_film': hist_film, 'hist_all': hist_all}
with open('lab06.json', 'wt') as file:
    file.writelines(json.dumps(response))

In [128]:
# Останавливаем контекст
sc.stop()